# Inference example for a model predicting creatinine thresholds

## Imports and utility functions

In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from strats_pytorch.datasets.dataset_int_classif import MIMIC_Int_Classif, padded_collate_fn
from strats_pytorch.models.strats import STraTS


def denorm(val, mean, std):
    return (val * std) + mean

## Model Initialization

In [ ]:
exp_n = 0
exp_path = f"exp_creatinine/exp_{exp_n}/"

model = STraTS(
    n_var_embs=17,
    dim_demog=2,
    dropout=0.0,
    n_layers=2,
    dim_embed=52,
    n_heads=4,
    forecasting=False,
    n_classes=4
)
model.load_state_dict(torch.load(exp_path + "STraTS.pth"))

## Dataset and Dataloader Initialization

In [ ]:
test_ds = MIMIC_Int_Classif(
    data_path="generated/creat17NoText_culled.csv"
)
test_ds.restrict_to_indexes(np.load(exp_path + "test_idx.npy"))

# Variable and Time normalization
test_ds.normalize(normalize_vars=True, normalize_times=True, verbose=True)


In [ ]:
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    collate_fn=padded_collate_fn, 
    shuffle=False # Note: we need shuffle False to know which time_mean and time_std to use for plotting
)

## Example time series

In [ ]:
# Get a single time series, of index stay_ind
# Note: we need the dataloader to generate the mask
stay_ind = 3
for ind, (demog, values, times, variables, target, mask) in enumerate(test_dl):
    if ind==stay_ind:
        break

# actual stay index just for time denormalization later when plotting
stay_ind = test_ds.indexes[stay_ind]

In [ ]:
demog_unbatched = demog[0]

gender = "Male" if demog_unbatched[0] == -1 else "Female"
age = denorm(demog_unbatched[1], test_ds.age_mean, test_ds.age_std)
print(f"Demog Info: {gender}, {age} y.o.")

### Plotting each variable

In [ ]:
values_unbatched = values[0]
times_unbatched = times[0]
variables_unbatched = variables[0]

# Collect the values and times per variable for plotting
vals_per_var = {}
times_per_var = {}
for i, v in enumerate(variables_unbatched):
    v = v.item()
    # init if needed
    if v not in vals_per_var.keys():
        vals_per_var[v] = []
        times_per_var[v] = []

    vals_per_var[v].append(denorm(values_unbatched[i], test_ds.means[v], test_ds.stds[v]))
    times_per_var[v].append(denorm(times_unbatched[i], test_ds.time_means[stay_ind], test_ds.time_stds[stay_ind]))

In [ ]:
ind_to_var = {
    0: "creat",
    1: "Heart rate",
    2: "BP systolic",
    3: "BP diastolic",
    4: "BP mean",
    5: "Temp F",
    6: "weight Daily",
    7: "weight Admisison",
    8: "White Blood Cell Count",
    9: "sodium (serum)",
    10: "potassium (serum)",
    11: "arterial pH",
    12: "respiratory rate",
    13: "apnea interval",
    14: "minute volume",
    15: "Central Venous Pressure",
    16: "O2 fraction",
}

plt.figure(figsize=(15, 12))
for i, var_id in enumerate(times_per_var.keys()):
    plt.subplot(6, 3, i+1)
    plt.plot(times_per_var[var_id], vals_per_var[var_id], "x-")
    plt.ylabel(ind_to_var[var_id])

plt.show()

## Model prediction

In [ ]:
pred = model(demog, values, times, variables, mask)
pred_label = pred.argmax().item()
print(f"Pred label: {pred_label}; Ground Truth:{int(target.item())}")